原文链接：[Async IO in Python](https://realpython.com/async-io-python/)

主要内容：

* 异步 IO：语言无关的编程范式，在多种语言中都有实现。
* async/await：Python 中的两个新关键字
* asyncio：Python 包，提供了运行和管理 coroutine 的 API。

协程是 asyncio 的核心，需要仔细了解。

# Async IO 概览

## Async IO 适合的场景

并发（Concurrency）和并行（Parallelism）是被广泛使用的术语。

* 并行：包含多个**同时执行**的操作。**多进程（Multiprocessing）**是实现并行的一种方式。多进程处理适用于CPU-bound的任务。
* 并发：比并行范围更广，它表示多个任务可以**同时存在（从而执行之）**，关键是同时存在并不真正同时执行，比如单核CPU的情形。
* 多线程：并发执行模型的一种。多线程适合IO-bound任务。

Python 标准库早已支持以上各种机制，通过 `multiprocessing`、 `threading` 和 `concurrent.futures` 这些包。

**Async IO（asyncio）**是较新的一个包，但在其它语言已有实现，如`Go`、`C#`等。

asyncio 的文档说它是一个编写并发代码的库，但它不是多进程的，甚至不是多线程的，看起来很神秘。它是单线程、单进程的设计：它使用**协作式多任务（cooperative multitasking）**。

还有另一个概念：**异步（asynchronous）**，它有两个性质：

* 异步例程可以“暂停”，等待其它结果到来，并让其它例程执行；
* 异步代码，通过上述机制，实现并发执行，换言之，异步代码是一种并发的实现方式。

## asyncio 详解

asyncio 乍看起来是反直觉的，毕竟单线程如何实现并发呢？下面这个解释或许有所帮助。

象棋大师波尔加同时与24名业余棋手下棋，假设她每步棋需要5秒，对手需要55秒，每盘棋走30个来回。她有同步和异步两种方式来走：

* 同步：每次只与一个人下棋，下完需要30分钟，总共需要12小时。
* 异步：波尔加逐桌下过去，每走完一步，就留给对手去走，最后需要的时间只是1小时左右。

两种方式，都是一个人走完所有的步数，但时间差了许多。关键在于，对手们等待的时间少了许多。

## asyncio 不容易

# asyncio 包与 `async/await`

## 协程

asyncio的内核乃是协程。协程是一种函数，它可以在return之前挂起执行，将执行传给其它协程。

In [ ]:
# coding=utf-8

import asyncio


async def count():
    print('one')
    await asyncio.sleep(1)
    print('two')


async def main():
    await asyncio.gather(count(), count(), count())


if __name__ == '__main__':
    import time
    start = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - start
    print(f'{__file__} executed in {elapsed:0.2f} seconds.')


这里的一个关键点是，`await asyncio.sleep` 是非阻塞调用，因此执行到这里后，其它函数可以继续执行。`await` 某个调用，将执行权转交给其它可以立即做事的程序。而 `time.sleep` 则是典型的阻塞调用。

## asyncio 规则

* `async def` 引入一个原生协程（native coroutine）或异步生成器（asynchronous generator）。
* `await` 将控制权返回event loop，后者挂起 await 语句所在的程序。

In [2]:
async def g():
    # Pause here and come back to g() when f() is ready
    r = await f()
    return r

`async/await`的使用范围亦有严格限定。

* 通过`async def`定义的函数是协程，它可以使用`await`、`return`或`yield`，但都是可选的。
    - `await`、`return`创建协程函数，要调用之，需要使用`await`获取其返回值；
    - `yield`创建异步生成器，可用于`yield for`。
    - 不可以使用`yield from`
* `async def`协程之外的地方不可以使用`await`。
* 使用`await f()`时，f需要是awaitable的，即要么是协程，要么是定义了`__await__`方法。

py 3.4 使用装饰器，3.5 使用 async/await，前者是 generator-based，后者是 native 的。

大部分程序包含小的、模块化的协程以及一个 wrapper 函数，wrapper 串联起各个协程，收集其结果。

In [ ]:
# coding=utf-8
import asyncio
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)


async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i


async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res


if __name__ == "__main__":
    random.seed(444)
    r1, r2, r3 = asyncio.run(main())
    print()
    print(f"r1: {r1}, r2: {r2}, r3: {r3}")

# Async IO 设计模式

## Chaining

协程的一个重要特点是可以链式使用，如此可将程序分解为更小的部分。

## 作为 Queue

asyncio 提供了 Queue 类，类似于 queue 模块中同名类。

除了上述例子中演示的情况，还有其它可能，若干 producer，彼此不相关，都会像一个 queue 中添加 item。同时有 consumer 从 queue 中 pull 条目，消费之，对 producer 有多少个，条目有几何，皆不知情。此时，queue 充当两者的协调者。

In [ ]:
# coding=utf-8
import asyncio
import itertools as it
import os
import random
import time


async def makeitem(size: int = 5) -> str:
    return os.urandom(size).hex()


async def randsleep(a: int = 1, b: int = 5, caller=None) -> None:
    i = random.randint(0, 10)
    if caller:
        print(f"{caller} sleeping for {i} seconds.")
    await asyncio.sleep(i)


async def produce(name: int, q: asyncio.Queue) -> None:
    n = random.randint(0, 10)
    for _ in it.repeat(None, n):  # Synchronous loop for each single producer
        await randsleep(caller=f"Producer {name}")
        i = await makeitem()
        t = time.perf_counter()
        await q.put((i, t))
        print(f"Producer {name} added <{i}> to queue.")


async def consume(name: int, q: asyncio.Queue) -> None:
    while True:
        await randsleep(caller=f"Consumer {name}")
        i, t = await q.get()
        now = time.perf_counter()
        print(f"Consumer {name} got element <{i}>"
              f" in {now-t:0.5f} seconds.")
        q.task_done()


async def main(nprod: int, ncon: int):
    q = asyncio.Queue()
    producers = [asyncio.create_task(produce(n, q)) for n in range(nprod)]
    consumers = [asyncio.create_task(consume(n, q)) for n in range(ncon)]
    await asyncio.gather(*producers)
    await q.join()  # Implicitly awaits consumers, too
    for c in consumers:
        c.cancel()

if __name__ == "__main__":
    import argparse
    random.seed(444)
    parser = argparse.ArgumentParser()
    parser.add_argument("-p", "--nprod", type=int, default=5)
    parser.add_argument("-c", "--ncon", type=int, default=10)
    ns = parser.parse_args()
    start = time.perf_counter()
    asyncio.run(main(**ns.__dict__))
    elapsed = time.perf_counter() - start
    print(f"Program completed in {elapsed:0.5f} seconds.")


## Async IO 源于 Generator

generator 的一个重要特征是，它可以停止，之后也可以重启。生成器之所以优于函数，概由于此。

In [3]:
from itertools import cycle

def endless():
    yield from cycle((9, 8, 7, 6))
    
e = endless()
print(e)

total = 0
for i in e:
    if total < 30:
        print(i, end=' ')
        total += i
    else:
        print()
        break
        
# resume
next(e), next(e)

<generator object endless at 0x11269bc50>
9 8 7 6 


(8, 7)

await 与此类似，它标记出一个breakpoint，挂起执行。

generator 另一个特征是，可以通过 `send` 方法发送一个值。

简言之：

* 协程是生成器的改头换面
* 基于生成器的协程使用 yield from，原生的则代之以 await
* await 可看作一个信号，标记出breakpoint，它让协程暂时挂起，运行程序晚点之后回来

## 其它特征：async for

async for 用于迭代一个异步迭代器，以及异步生成器、异步推导。

异步推导，并非以并发方式去 map 一个 iterable。事实上，`async for` 与 `async with` 使用时都是因为，`for` 与 `with` 会打破 `await`的特征。

## Event Loop 与 asyncio.run()

事件循环监控协程们的运行，接收到某些协程 idle 的反馈，寻找可在同时执行的其它任务。当某个协程拿到 wait 的结果时，使其继续执行。

目前的用法都是：

```python
asyncio.run(main())
```

更老的一种是：

In [ ]:
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
finally:
    loop.close()

# Async IO 何时使用

async io 与多进程并非直接”对手“，实际上他们可以结合使用。如果有多个较为均匀的 CPU-bound 任务，那么多进程是不二之选。另外，如果函数都是阻塞式调用，那么一概添加 aysnc 也毫无益处。

至于多线程，两者对比更为明显。线程属于系统资源，扩展性有限。在一台机器上，创建数千个 async io 任务是 feasible 的。

`async io`适用的地方是，有多个IO-bound的任务，如果不使用，会导致大量的 IO-bound 等待时间，如：

* Network IO
* Serverless designs，如 group chatroom
* 读写操作，你希望模拟”fire-and-forget“风格，但不希望担心各种”lock“问题。

而它不适用的地方是，它仅支持有限的一组方法。如果要访问某个DBMS，那么需要一个新的支持 async/await 的 wrapper，否则包含同步调用的协程会极大的阻塞其它协程的执行。